In [1]:
from langchain_groq import ChatGroq
from secret_key import grok_key

llm = ChatGroq(
    #model = "deepseek-r1-distill-llama-70b",
    model = 'llama-3.3-70b-versatile',
    groq_api_key = grok_key,
    temperature=0,
    max_tokens = None,
    timeout = None,
    max_retries = 2,
)

response = llm.invoke('The first person to walk on the moon was....')
print(response.content)

The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [6]:
from  langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://careers.nike.com/nike-stores-franchise-partner-manager/job/R-64538')
page_data= loader.load().pop().page_content
print(page_data)






















Nike Stores Franchise Partner Manager










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previ

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract he job posting and return them in JSON format containing the 
    following keys: 'role','experience','skills',and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)"""
)

chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'page_data':page_data})
print(response.content)

```json
{
  "role": "Nike Stores Franchise Partner Manager",
  "experience": "Experience in managing Partners, driving fleet development, understanding retail KPIs and driving actions x-functionally to improve productivity and profitability of Partner stores",
  "skills": [
    "Strong leadership skills",
    "X-functional team and stakeholders management skills",
    "Knowledge of the marketplace and understanding of real estate dynamics",
    "Savyness in business and retail KPIs",
    "Contract Management: negotiations, renewals, adjustments",
    "Retail & Operations acumen",
    "Strong communication and presentation skills",
    "Business savviness: ability to understand retail KPIs and productivity/profitability results, performance management and ability to identify commercial opportunities to drive sales"
  ],
  "description": "We’re looking for a highly motivated individual to join our team and be the Primary Lead of Partner Business in France, ensuring Strategic Alignment an

In [10]:
type(response.content) # it is in string format .. we have to convert it into json format

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
json_response

{'role': 'Nike Stores Franchise Partner Manager',
 'experience': 'Experience in managing Partners, driving fleet development, understanding retail KPIs and driving actions x-functionally to improve productivity and profitability of Partner stores',
 'skills': ['Strong leadership skills',
  'X-functional team and stakeholders management skills',
  'Knowledge of the marketplace and understanding of real estate dynamics',
  'Savyness in business and retail KPIs',
  'Contract Management: negotiations, renewals, adjustments',
  'Retail & Operations acumen',
  'Strong communication and presentation skills',
  'Business savviness: ability to understand retail KPIs and productivity/profitability results, performance management and ability to identify commercial opportunities to drive sales'],
 'description': 'We’re looking for a highly motivated individual to join our team and be the Primary Lead of Partner Business in France, ensuring Strategic Alignment and Business Development Plans, and ob

In [11]:
type(json_response)

dict

In [12]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [17]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                      metadatas={'links':row['Links']},
                      ids = [str(uuid.uuid4())])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [20]:
links = collection.query(query_texts =['Experience in python','Extertise in React Native'],n_results = 2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [21]:
job = json_response
job['skills']

['Strong leadership skills',
 'X-functional team and stakeholders management skills',
 'Knowledge of the marketplace and understanding of real estate dynamics',
 'Savyness in business and retail KPIs',
 'Contract Management: negotiations, renewals, adjustments',
 'Retail & Operations acumen',
 'Strong communication and presentation skills',
 'Business savviness: ability to understand retail KPIs and productivity/profitability results, performance management and ability to identify commercial opportunities to drive sales']

In [22]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION
    {job_description}
    
    ### INSTRUCTION:
    You are Harry, a Data Scientist who is looking for an internship or a job.
    You have descent experience in all the tools and like python , pandas, numpy, tensoflow, machine learning, deeplearnin, and more
    Your job is to write a cold email to the HR regarding the job mentioned above describing the capability of your skills and knowledge
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Your protfolio: {link_list}
    Remember you are Harry,Data Scientist Fresher.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
response = chain_email.invoke({"job_description":str(job),"link_list":links})
print(response.content)


Subject: Application for Nike Stores Franchise Partner Manager Role

Dear Hiring Manager,

I am excited to apply for the Nike Stores Franchise Partner Manager position, as advertised. Although my background is in Data Science, I am confident that my skills and knowledge can be adapted to fulfill the requirements of this role.

As a detail-oriented and analytical individual with experience in managing projects and collaborating with cross-functional teams, I possess strong leadership skills. My proficiency in Python, pandas, and numpy has equipped me with the ability to analyze and interpret complex data, which can be applied to understanding retail KPIs and driving business decisions. Additionally, my knowledge of machine learning and deep learning can be utilized to identify commercial opportunities and drive sales.

In terms of contract management, I have experience with negotiations and adjustments through various projects and collaborations. My strong communication and presentation